In [5]:
from bs4 import BeautifulSoup
import subprocess
import pandas as pd
from funcs import tratamento_texto

In [ ]:
def extrair_eventos(ano_inicio, ano_fim):
  data = []
  for ano in range(ano_inicio, ano_fim+1):
    url = "https://www.onthisday.com/date" if ano < 1492 else "https://www.onthisday.com/events/date"
    for pagina in range(1, 6):
      command = f"{url}/{ano}?p={pagina}" if pagina >= 2 else f"{url}/{ano}"
      result = subprocess.run(["curl", command], capture_output=True, text=True, encoding='utf-8')
      soup = BeautifulSoup(result.stdout, 'html.parser')
      try:
        year = soup.find('span', class_='year').get_text()
        highlights = soup.find_all('div', class_='section--highlight')
        for hl in highlights:
          date = hl.find('p').a.get_text()  
          event = hl.get_text().replace(date, '', 1).strip()
          data.append({"Year": year, "Date": date, "Event": event})
        article = soup.find('article')
        event_list = article.find_all('li', class_=['event', 'person'])
        for e in event_list:
          date = None
          if e.b:
            date = e.b.get_text()  
          for cls in ['date', 'deathDate', 'birthDate']:
            tag = e.find('a', class_=cls)
            if tag:
              date = tag.get_text()
          event = e.get_text().replace(date, '', 1).strip()
          data.append({"Year": year, "Date": date, "Event": event})
      except:
        break
  return data

In [ ]:
df = pd.DataFrame(extrair_eventos(1, 2024))
df

,Year,Date,Event
0,1,Jan 1,Origin of the Christian Era
1,1,Mar 25,Origin of Dionysian Incarnation of the Word
2,1,Dec 25,The first Christmas according to calendar-make...
3,2,Aug 20,Venus and Jupiter in conjunction - possible as...
4,3,Aug 12,Venus-Jupiter in conjunction-Star of Bethlehem
...,...,...,...
77992,2024,Dec 24,NASA's Parker Solar probe makes a record-break...
77993,2024,Dec 24,Pope Francis opens the Holy Door of St Peter's...
77994,2024,Dec 25,Azerbaijani airliner crashes near Kazakhstani ...
77995,2024,Dec 29,South Korean Jeju Air plane crashes and explod...


In [1]:
coluna_texto = "Event"

In [7]:
df["texto_tratado"] = df[coluna_texto].apply(tratamento_texto)

In [8]:
df

,Year,Date,Event,texto_tratado
0,1,Jan 1,Origin of the Christian Era,origin christian era
1,1,Mar 25,Origin of Dionysian Incarnation of the Word,origin dionysian incarnation word
2,1,Dec 25,The first Christmas according to calendar-make...,first christmas according calendarmaker easter...
3,2,Aug 20,Venus and Jupiter in conjunction - possible as...,venus jupiter conjunction possible astrologica...
4,3,Aug 12,Venus-Jupiter in conjunction-Star of Bethlehem,venusjupiter conjunctionstar bethlehem
...,...,...,...,...
77992,2024,Dec 24,NASA's Parker Solar probe makes a record-break...,nasas parker solar probe makes recordbreaking ...
77993,2024,Dec 24,Pope Francis opens the Holy Door of St Peter's...,pope francis opens holy door st peters basilic...
77994,2024,Dec 25,Azerbaijani airliner crashes near Kazakhstani ...,azerbaijani airliner crashes near kazakhstani ...
77995,2024,Dec 29,South Korean Jeju Air plane crashes and explod...,south korean jeju air plane crashes explodes t...


In [9]:
df.to_csv("dataset/events.csv", sep=";", index=False)